In [1]:
import sys


In [2]:
print(sys.version)


3.5.4 (default, Aug  8 2017, 11:09:21) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-11)]


In [3]:
import os
from PIL import Image
import cv2
import keras as ks
from keras import backend as K
K.set_image_data_format( 'channels_last' ) 
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.cross_validation import train_test_split


Using TensorFlow backend.
/usr/lib64/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
DatasetPath=[]
imageLabels=[]
k=-1
for i in os.listdir("faces/"):
    k=k+1
    for j in os.listdir("faces/"+i):
        DatasetPath.append(os.path.join("faces/",i,j))
        imageLabels.append(k)
print (k)

6


In [5]:
imageData=[]
k=-1
for i in DatasetPath:
    imgRead=cv2.imread(i)
    imgRead=cv2.resize(imgRead,(224,224))
    imageData.append(imgRead)

In [6]:
print (len(imageData))
print (len(imageLabels))

140
140


In [7]:
imageData[1].shape

(224, 224, 3)

In [8]:
from keras.utils import np_utils

In [9]:
(X_train, X_test,y_train,y_test)=train_test_split(np.array(imageData),np.array(imageLabels),train_size=0.9,random_state=12)

In [10]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [11]:
X_test.shape

(14, 224, 224, 3)

In [12]:
nb_classes=7

In [13]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [14]:
Y_test=np_utils.to_categorical(y_test,nb_classes)

In [15]:
Y_train=np_utils.to_categorical(y_train,nb_classes)

# X_train = X_train.reshape(2034, 224*224)
X_test = X_test.reshape(226, 224*224)

# X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

# print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

In [16]:
import numpy as np


#%matplotlib inline
#import matplotlib.pyplot as plt

In [17]:
def convblock(cdim, nb, bits=3):
    L = []
    
    for k in range(1,bits+1):
        convname = 'conv'+str(nb)+'_'+str(k)
        #L.append( Convolution2D(cdim, 3, 3, border_mode='same', activation='relu', name=convname) ) # Keras 1
        L.append( Convolution2D(cdim, kernel_size=(3, 3), padding='same', activation='relu', name=convname) ) # Keras 2
    
    L.append( MaxPooling2D((2, 2), strides=(2, 2)) )
    
    return L

In [18]:
def vgg_face_blank(nb_classes):
    
    withDO = True # no effect during evaluation but usefull for fine-tuning
    
    if True:
        mdl = Sequential()
        
        # First layer is a dummy-permutation = Identity to specify input shape
        mdl.add( Permute((1,2,3), input_shape=(224,224,3)) ) # WARNING : 0 is the sample dim

        for l in convblock(64, 1, bits=2):
            mdl.add(l)

        for l in convblock(128, 2, bits=2):
            mdl.add(l)
        
        for l in convblock(256, 3, bits=3):
            mdl.add(l)
            
        for l in convblock(512, 4, bits=3):
            mdl.add(l)
            
        for l in convblock(512, 5, bits=3):
            mdl.add(l)
        
        #mdl.add( Convolution2D(4096, 7, 7, activation='relu', name='fc6') ) # Keras 1
        mdl.add( Convolution2D(4096, kernel_size=(7, 7), activation='relu', name='fc6') ) # Keras 2
        if withDO:
            mdl.add( Dropout(0.5) )
        #mdl.add( Convolution2D(4096, 1, 1, activation='relu', name='fc7') ) # Keras 1
        mdl.add( Convolution2D(4096, kernel_size=(1, 1), activation='relu', name='fc7') ) # Keras 2
        if withDO:
            mdl.add( Dropout(0.5) )
        #mdl.add( Convolution2D(2622, 1, 1, name='fc8') ) # Keras 1
        mdl.add( Convolution2D(nb_classes, kernel_size=(1, 1), activation='relu', name='fc8') ) # Keras 2
        mdl.add( Flatten() )
        mdl.add( Activation('softmax') )
        
        return mdl
    
    else:
        # See following link for a version based on Keras functional API :
        # gist.github.com/EncodeTS/6bbe8cb8bebad7a672f0d872561782d9
        raise ValueError('not implemented')

In [19]:
facemodel = vgg_face_blank(2622)
facemodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
__________

In [20]:
from scipy.io import loadmat

In [21]:
data = loadmat('vgg-face.mat', matlab_compatible=False, struct_as_record=False)
l = data['layers']
description = data['meta'][0,0].classes[0,0].description


In [22]:
l.shape, description.shape

((1, 37), (2622, 1))

In [23]:
l[0,10][0,0].type[0], l[0,10][0,0].name[0]

('conv', 'conv3_1')

In [24]:
l[0,10][0,0].weights[0,0].shape, l[0,10][0,0].weights[0,1].shape

((3, 3, 128, 256), (256, 1))

In [25]:
def weight_compare(kmodel):
    kerasnames = [lr.name for lr in kmodel.layers]

    # WARNING : important setting as 2 of the 4 axis have same size dimension
    #prmt = (3,2,0,1) # INFO : for 'th' setting of 'dim_ordering'
    prmt = (0,1,2,3) # INFO : for 'channels_last' setting of 'image_data_format'

    for i in range(l.shape[1]):
        matname = l[0,i][0,0].name[0]
        mattype = l[0,i][0,0].type[0]
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            print(matname, mattype)
            print(l[0,i][0,0].weights[0,0].transpose(prmt).shape, l[0,i][0,0].weights[0,1].shape)
            print(kmodel.layers[kindex].get_weights()[0].shape, kmodel.layers[kindex].get_weights()[1].shape)
            print('------------------------------------------')
        else:
            print('MISSING : ', matname, mattype)
            print('------------------------------------------')

In [26]:
def copy_mat_to_keras(kmodel):

    kerasnames = [lr.name for lr in kmodel.layers]

    # WARNING : important setting as 2 of the 4 axis have same size dimension
    #prmt = (3,2,0,1) # INFO : for 'th' setting of 'dim_ordering'
    prmt = (0,1,2,3) # INFO : for 'channels_last' setting of 'image_data_format'

    for i in range(l.shape[1]):
        matname = l[0,i][0,0].name[0]
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            #print matname
            l_weights = l[0,i][0,0].weights[0,0]
            l_bias = l[0,i][0,0].weights[0,1]
            f_l_weights = l_weights.transpose(prmt)
            #f_l_weights = np.flip(f_l_weights, 2) # INFO : for 'th' setting in dim_ordering
            #f_l_weights = np.flip(f_l_weights, 3) # INFO : for 'th' setting in dim_ordering
            assert (f_l_weights.shape == kmodel.layers[kindex].get_weights()[0].shape)
            assert (l_bias.shape[1] == 1)
            assert (l_bias[:,0].shape == kmodel.layers[kindex].get_weights()[1].shape)
            assert (len(kmodel.layers[kindex].get_weights()) == 2)
            kmodel.layers[kindex].set_weights([f_l_weights, l_bias[:,0]])
            #print '------------------------------------------'

In [27]:
copy_mat_to_keras(facemodel)

# def pred(kmodel, crpimg, transform=False):
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940
        #
        # WARNING : in this script (https://github.com/rcmalli/keras-vggface) colours are switched
        aux = copy.copy(imarr)
        #imarr[:, :, 0] = aux[:, :, 2]
        #imarr[:, :, 2] = aux[:, :, 0]

        #imarr[:,:,0] -= 129.1863
        #imarr[:,:,1] -= 104.7624
        #imarr[:,:,2] -= 93.5940

    #imarr = imarr.transpose((2,0,1)) # INFO : for 'th' setting of 'dim_ordering'
    imarr = np.expand_dims(imarr, axis=0)

    out = kmodel.predict(imarr)

    best_index = np.argmax(out, axis=1)[0]
    best_name = description[best_index,0]
    print(best_index, best_name[0], out[0,best_index], [np.min(out), np.max(out)])


# im = Image.open('Downloads/ak.png') # WARNING : this image is well centered and square
im = im.resize((224,224))

# %ls

# plt.imshow(np.asarray(im))


# crpim = im

# import copy
pred(facemodel, crpim, transform=False)
pred(facemodel, crpim, transform=True)


# description[102,0][0]


# description

In [28]:
layer_count = 0
for layer in facemodel.layers:
	layer_count = layer_count+1
for l in range(layer_count-3):
	facemodel.layers[l].trainable=False


In [29]:
layer_count
facemodel.layers.pop()
facemodel.layers.pop()
facemodel.layers.pop()

In [30]:
facemodel.outputs = [facemodel.layers[-1].output]

In [31]:
facemodel.layers[-1]

In [32]:
facemodel.add( Convolution2D(7, kernel_size=(1, 1), activation='relu') ) # Keras 2
facemodel.add( Flatten() )
facemodel.add( Activation('softmax') )

In [33]:

facemodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
__________

# facemodel.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

# optimizer.lr.set_value(0.02)

# facemodel.optimizer.lr.set_value(0.01)

# sgd = SGD(lr=0.1, decay=0, momentum=0.9, nesterov=True)

# opti=facemodel.optimizer

# opti.lr.get_value()

# from keras.callbacks import LearningRateScheduler

# def scheduler(epoch):
    lr = K.get_value(facemodel.optimizer.lr)
    print(lr)
    K.set_value(facemodel.optimizer.lr, 0.01)
    return K.get_value(facemodel.optimizer.lr)

# lr_decay = LearningRateScheduler(scheduler)

In [33]:
adam = ks.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# facemodel.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [34]:
facemodel.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [35]:
from time import time
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir="logs/final/{}".format(time()), histogram_freq=1, write_graph=True, write_images=True)


In [ ]:
facemodel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30, batch_size=100, verbose=1,callbacks=[tensorboard])

Train on 126 samples, validate on 14 samples
Epoch 1/30
126/126 [==============================] - 66s - loss: 5.0479 - acc: 0.1270 - val_loss: 1.8153 - val_acc: 0.4286
Epoch 2/30
126/126 [==============================] - 67s - loss: 2.1370 - acc: 0.4444 - val_loss: 0.6681 - val_acc: 0.7857
Epoch 3/30
126/126 [==============================] - 67s - loss: 0.8985 - acc: 0.6349 - val_loss: 0.3428 - val_acc: 0.8571
Epoch 4/30
126/126 [==============================] - 67s - loss: 0.5765 - acc: 0.7937 - val_loss: 0.2876 - val_acc: 0.8571
Epoch 5/30
126/126 [==============================] - 67s - loss: 0.4562 - acc: 0.8254 - val_loss: 0.2796 - val_acc: 0.8571
Epoch 6/30
126/126 [==============================] - 67s - loss: 0.3031 - acc: 0.8571 - val_loss: 0.2783 - val_acc: 0.8571
Epoch 7/30
126/126 [==============================] - 67s - loss: 0.3357 - acc: 0.8333 - val_loss: 0.2781 - val_acc: 0.8571
Epoch 8/30
126/126 [==============================] - 68s - loss: 0.2815 - acc: 0.8571 

In [ ]:
model = Sequential()
model.add(Dense(512,input_shape=(X_train.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
adam = ks.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=64, nb_epoch=50, verbose=1, validation_data=(X_test, Y_test))